In [1]:
!pip install scikeras

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN
from keras import optimizers
from scikeras.wrappers import KerasClassifier, KerasRegressor

In [62]:
df = pd.read_csv('sentiment_analysis_dataset.csv', error_bad_lines = False)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

<ipython-input-62-911dc28f874d>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('sentiment_analysis_dataset.csv', error_bad_lines = False)


,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [63]:
df = df.dropna()

In [64]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [65]:
x = df['text']
y = df['sentiment']

In [66]:
y = pd.get_dummies(y, columns = ['sentiment'])
y

,negative,neutral,positive
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
12507,1,0,0
12508,1,0,0
12509,0,0,1
12510,0,1,0


In [83]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 84)

In [84]:
x_train = [str(sample) for sample in x_train]
x_test = [str(sample) for sample in x_test]

In [85]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

In [86]:
maxlen = 100

x_train = pad_sequences(x_train, padding = 'post', maxlen = maxlen)
x_test = pad_sequences(x_test, padding = 'post', maxlen = maxlen)

In [87]:
num_classes = 2
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [88]:
x_train = np.array(x_train).reshape((x_train.shape[0], x_train.shape[1], 1))
print(x_train.shape)

x_test = np.array(x_test).reshape((x_test.shape[0], x_test.shape[1], 1))
print(x_test.shape)

(9383, 100, 1)
(3128, 100, 1)


In [89]:
def basic_RNN():
  model = Sequential()
  model.add(SimpleRNN(50, input_shape = (maxlen, 1)))
  model.add(Dense(6))
  model.add(Activation('softmax'))
  model.summary()

  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

  return model

In [91]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
y_train_reshaped = y_train.reshape(y_train.shape[0], -1)

print(x_train_reshaped.shape)
print(y_train_reshaped.shape)

(9383, 100)
(9383, 6)


In [79]:
model = KerasClassifier(build_fn = basic_RNN, epochs = 30, batch_size = 50)
model.fit(x_train_reshaped, y_train_reshaped)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_6 (Dense)             (None, 6)                 306       
                                                                 
 activation_6 (Activation)   (None, 6)                 0         
                                                                 
Total params: 2906 (11.35 KB)
Trainable params: 2906 (11.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
188/188 [==============================] - 5s 23ms/step - loss: 5.4836 - accuracy: 0.2535
Epoch 2/30
188/188 [==============================] - 3s 18ms/step - loss: 5.4295 - accuracy: 0.3300
Epoch 3/30
188/188 [==============================] - 3s 17ms/step - loss: 5.3869 - accuracy: 0.3496
Epoch 4/30
188/188 [==============================] - 4s 22ms/step - loss: 5.3929 - accuracy: 0.3489
Epoch 5/30
188/188 [==============================] - 6s 34ms/step - loss: 5.3335 - accuracy: 0.3547
Epoch 6/30
188/188 [==============================] - 9s 46ms/step - loss: 5.3693 - accuracy: 0.3538
Epoch 7/30
188/188 [==============================] - 5s 25ms/step - loss: 5.3292 - accuracy: 0.3803
Epoch 8/30
188/188 [==============================] - 4s 20ms/step - loss: 5.3136 - accuracy: 0.4027
Epoch 9/30
188/188 [==============================] - 4s 23ms/step - loss: 5.3611 - accuracy: 0.3540
Epoch 10/30
188/188 [==============================] - 4s 19ms/step - loss: 5.3469 - accura

KerasClassifier(
	model=None
	build_fn=<function basic_RNN at 0x7a62b75b3b50>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=50
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=30
	class_weight=None
)

In [101]:
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
y_test_reshaped = y_test.reshape(y_test.shape[0], -1)
y_test_reshaped = pd.DataFrame(y_test_reshaped)
x_test_reshaped = pd.DataFrame(x_test_reshaped)

In [ ]:
score = model.score(x_test_reshaped, y_test_reshaped)
print("Test Score:", score)